<a href="https://colab.research.google.com/github/DSKI23/alzheimervalentin/blob/cleanup/Modell/logged-hyperparameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r "https://raw.githubusercontent.com/DSKI23/alzheimervalentin/refs/heads/main/requirements_colab.txt"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.6/391.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.8/325.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.2/74.2 kB 4.6 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login
from datasets import load_dataset
import wandb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
import pickle
import os

In [3]:
notebook_login()

In [ ]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

In [ ]:
dataset = load_dataset("DS23-KI-Projekt/alzheimerdataset_split")

df = dataset['train'].to_pandas()

df.columns

In [ ]:
# Function to load dataset
def load_data():
    try:
        df = dataset["train"].to_pandas()
        target = "Alzheimer’s Diagnosis"
        features = [col for col in df.columns if col != target]
        return df[features], df[target]
    except Exception as e:
        print(f"Error loading dataset: {e}")
        exit(1)

# Function to train and evaluate Random Forest
def train_rf(X_train, X_test, y_train, y_test, n_estimators):
    rf_model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)
    rf_model.fit(X_train, y_train)

    y_pred = rf_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    class_report = classification_report(y_test, y_pred, output_dict=True)  # JSON format for WandB
    cfm = confusion_matrix(y_test, y_pred).tolist()  # Convert to list for WandB logging

    return rf_model, accuracy, class_report, cfm

#  Main function
def main():
    # Initialize WandB
    wandb.init(entity="zhannalialko-dhbw-mosbach", project="alzheimer-rf", name="RF-Training")

    # Load dataset
    X, y = load_data()

    # Split dataset (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

    best_model = None
    best_accuracy = 0

    #  Train over 10 epochs with increasing estimators
    for epoch in range(1, 11):
        n_estimators = epoch * 10  # Start with 10 trees, increase to 100

        rf_model, accuracy, class_report, cfm = train_rf(X_train, X_test, y_train, y_test, n_estimators)

        # Save best model
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = rf_model

        # Log results in WandB
        wandb.log({
            "epoch": epoch,
            "accuracy": accuracy,
            "n_estimators": n_estimators,
            "classification_report": class_report,
            "confusion_matrix": cfm
        })

        print(f"Epoch {epoch} | n_estimators={n_estimators} | Accuracy={accuracy:.4f}")

    #  Save the best model
    model_dir = "random_forest_alzheimer"
    os.makedirs(model_dir, exist_ok=True)

    with open(f"{model_dir}/best_rf_model.pkl", "wb") as file:
        pickle.dump(best_model, file)

    print(f"Best model saved with accuracy: {best_accuracy:.4f}")

    # Finish WandB logging
    wandb.finish()

#  Run the script
if __name__ == "__main__":
    main()

Epoch 1 | n_estimators=10 | Accuracy=0.6820
Epoch 2 | n_estimators=20 | Accuracy=0.6946
Epoch 3 | n_estimators=30 | Accuracy=0.6959
Epoch 4 | n_estimators=40 | Accuracy=0.6977
Epoch 5 | n_estimators=50 | Accuracy=0.6974
Epoch 6 | n_estimators=60 | Accuracy=0.6980
Epoch 7 | n_estimators=70 | Accuracy=0.6990
Epoch 8 | n_estimators=80 | Accuracy=0.6986
Epoch 9 | n_estimators=90 | Accuracy=0.6987
Epoch 10 | n_estimators=100 | Accuracy=0.6987
Best model saved with accuracy: 0.6990


accuracy,▁▆▇▇▇█████
epoch,▁▂▃▃▄▅▆▆▇█
n_estimators,▁▂▃▃▄▅▆▆▇█
accuracy,0.69872
epoch,10
n_estimators,100
